# YOLOv8 Object Detection Example

## 1. Introduction

This notebook demonstrates a complete workflow for object detection using YOLOv8, a state-of-the-art, real-time object detection model. We will cover the essential steps from setting up the environment to performing inference with a fine-tuned model.

**Dataset:** We will use the "Objects Medical Supplies" dataset from Roboflow, which is a subset of PASCAL VOC images, focused on medical-related items. This dataset is conveniently available in YOLO format.

**Steps Covered:**
1.  **Setup:** Installing necessary libraries (`ultralytics` for YOLOv8, `PyTorch`, `OpenCV`).
2.  **Data Preparation:** Downloading the dataset, understanding its structure, and preparing the `data.yaml` configuration file required for YOLOv8 training.
3.  **Model Training (Fine-tuning):** Loading a pre-trained YOLOv8 model and fine-tuning it on our custom dataset.
4.  **Inference and Visualization:** Using the fine-tuned model to detect objects in new images and visualizing the results.

## 2. Setup: Install Libraries

In [ ]:
# Install Ultralytics for YOLOv8
# This package contains the YOLOv8 model implementation and utilities.
!pip install ultralytics

# Install PyTorch and torchvision
# YOLOv8 is built on PyTorch. Torchvision provides useful image transformation tools.
!pip install torch torchvision

# Install OpenCV for image processing
# OpenCV is used for image loading, manipulation, and displaying results.
!pip install opencv-python-headless

The code cell above installs the essential packages: `ultralytics` for YOLOv8, `torch` and `torchvision` as its deep learning framework, and `opencv-python-headless` for image processing tasks. Using `opencv-python-headless` is often preferred in server or container environments where GUI features are not needed.

## 3. Download and Prepare Dataset

The PASCAL VOC (Visual Object Classes) dataset is a widely recognized benchmark for object detection. For this notebook, we use a smaller, more specialized subset called "Objects Medical Supplies" from Roboflow. This allows for quicker demonstration of the training and inference pipeline. The dataset is provided in YOLO format, which simplifies the preparation steps.

In [ ]:
# Download the dataset (Medical Supplies - a PASCAL VOC subset from Roboflow)
# The dataset is already conveniently in YOLO format.
# The '-L' flag for curl follows redirects.
!curl -L "https://app.roboflow.com/ds/HzLEpSGavy?key=9B2P1CLD5P" > roboflow.zip

# Unzip the dataset
# The '-q' flag makes unzip operate quietly.
# The '-d VOC_subset' extracts files into the 'VOC_subset' directory.
!unzip -q roboflow.zip -d VOC_subset

# Remove the zip file after extraction to save space
!rm roboflow.zip 

# List the contents of the dataset directory to understand its structure
!ls VOC_subset

The dataset has been downloaded and unzipped into the `VOC_subset` directory. 
Typically, a YOLO formatted dataset contains:
- `train/`: Directory with training images and corresponding label files.
- `valid/`: Directory with validation images and corresponding label files.
- `test/` (optional): Directory with test images and corresponding label files.
- Each image file (e.g., `image.jpg`) has a corresponding text file (`image.txt`) in a `labels` subdirectory, where each line in the text file defines a bounding box (`class_id x_center y_center width height`).
- `data.yaml`: A configuration file that specifies paths to train/val/test sets, number of classes, and class names. The Roboflow download includes this file.

### Purpose of `data.yaml`

The `data.yaml` file is crucial for YOLOv8 as it provides the model with essential information about the dataset:

- `train`: Path to the directory containing training images (or a text file listing training images).
- `val`: Path to the directory containing validation images (or a text file listing validation images).
- `test` (optional): Path to the directory containing test images.
- `nc`: The total number of distinct object classes in the dataset.
- `names`: A list of strings, where each string is the name of an object class. The order of names corresponds to their class IDs (0 to `nc-1`).

While Roboflow provides a `data.yaml`, we will inspect it and then create our own `custom_data.yaml`. This ensures that the paths are correctly specified relative to our notebook environment (using absolute paths is a robust way to handle this, especially in Colab or similar platforms).

In [ ]:
import yaml
import os

# Path to the data.yaml file provided by Roboflow within the unzipped dataset
roboflow_yaml_path = 'VOC_subset/data.yaml'

# Initialize variables for class count and names
nc = 0
names = []

if os.path.exists(roboflow_yaml_path):
    # Open and load the YAML file from Roboflow
    with open(roboflow_yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
        print("--- Content of Roboflow's data.yaml ---")
        print(yaml.dump(data_config))
        
        # Extract the number of classes (nc) and class names
        nc = data_config['nc']
        names = data_config['names']
else:
    # Error message if Roboflow's YAML is not found (should not happen with this dataset link)
    print(f"Error: {roboflow_yaml_path} not found! Cannot automatically determine class names and number.")
    print("Please ensure the dataset downloaded and extracted correctly.")

# Define paths for our custom data.yaml. 
# These paths should point to the image directories within 'VOC_subset'.
# Using os.path.abspath ensures that YOLO can find the data regardless of the script's CWD.
base_dir = os.getcwd() # Current working directory of the notebook
train_images_path = os.path.join(base_dir, 'VOC_subset/train/images')
val_images_path = os.path.join(base_dir, 'VOC_subset/valid/images')
test_images_path = os.path.join(base_dir, 'VOC_subset/test/images') 

# Create the dictionary for our custom data.yaml
custom_data_yaml_content = {
    'train': train_images_path,
    'val': val_images_path,
    'nc': nc,
    'names': names
}

# Add the test set path if the directory exists and is not empty
if os.path.exists(test_images_path) and os.listdir(test_images_path):
    custom_data_yaml_content['test'] = test_images_path
else:
    # Print a message if the test set is not found or empty
    print(f"Test images directory '{test_images_path}' not found or is empty, omitting from custom_data.yaml.")

# Define the name for our custom YAML file
output_yaml_path = 'custom_data.yaml'

# Write the custom_data.yaml file
with open(output_yaml_path, 'w') as f:
    yaml.dump(custom_data_yaml_content, f, sort_keys=False, default_flow_style=None)

print(f"\n--- Content of created {output_yaml_path} ---")
with open(output_yaml_path, 'r') as f:
    print(f.read())

The code cell above performs two key actions:
1.  It inspects the `data.yaml` file that came with the Roboflow dataset to extract the number of classes (`nc`) and the list of class `names`.
2.  It then creates a new `custom_data.yaml` file in the notebook's root directory. This new YAML file uses absolute paths for the `train`, `val`, and (if available) `test` image directories. Using absolute paths (via `os.path.abspath` or `os.path.join(os.getcwd(), ...)`) is a good practice because it makes the dataset configuration robust to changes in the current working directory, which can sometimes occur in notebook environments or complex scripts.

## 4. Model Training (Fine-tuning)

In [ ]:
from ultralytics import YOLO
import os # os is already imported, but good to have it here if this cell is run independently

# Load a pre-trained YOLOv8 model.
# 'yolov8n.pt' is the nano version, which is small and fast, suitable for quick demonstrations.
# Other versions like 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt', 'yolov8x.pt' offer higher accuracy at the cost of size and speed.
# Using a '.pt' file loads pre-trained weights. Using a '.yaml' file (e.g., 'yolov8n.yaml') would build the model from scratch based on its architecture definition.
model = YOLO('yolov8n.pt') 

# Display model information (optional, but useful for verification)
# This shows the model architecture, number of layers, parameters, and GFLOPs.
model.info()

We are loading `yolov8n.pt`, which is a YOLOv8 nano model pre-trained on the COCO dataset. By using a pre-trained model, we leverage **transfer learning**.

**Benefits of Transfer Learning:**
- **Faster Training:** The model has already learned general visual features (edges, textures, basic shapes) from a large dataset. This allows it to learn the specifics of our new, smaller dataset much faster.
- **Less Data Required:** Fine-tuning often achieves good results with less data than training a model from scratch, as the foundational feature extraction capabilities are already present.
- **Potentially Better Performance:** Pre-trained models provide a strong starting point, which can lead to better overall performance, especially when the custom dataset is small or similar in nature to the pre-training dataset.

In [ ]:
# Define the path to the custom_data.yaml file created in the previous step.
# This file is located in the root directory of our notebook environment.
data_yaml_path = 'custom_data.yaml' 

# Train (fine-tune) the model on our custom dataset.
results = model.train(
    data=data_yaml_path,  # Path to our custom_data.yaml file.
    epochs=25,            # Number of training epochs. Keep low for a quick demo (e.g., 25-50). For production, this would be higher (e.g., 100-300).
    imgsz=640,            # Input image size. Images will be resized to 640x640 pixels.
    batch=8,              # Batch size. Adjust based on available GPU memory. Common values are 8, 16, 32.
    name='yolov8n_medical_supplies_finetuned', # Name for the training run. Results will be saved in 'runs/detect/yolov8n_medical_supplies_finetuned'.
    patience=5            # Early stopping patience. Training will stop if no improvement in validation metrics is observed for 5 consecutive epochs.
)

The `model.train()` function initiates the fine-tuning process. 

**Key Training Parameters Explained:**
- `data`: Specifies the path to our `custom_data.yaml` file, which defines the dataset structure and class information.
- `epochs`: The number of complete passes through the entire training dataset. For this demonstration, we use 25 epochs. In real-world scenarios, this value might be significantly higher (e.g., 100, 200, or more), depending on the dataset's size and complexity, and monitored by observing validation metrics.
- `imgsz`: The input image size (height and width) for the model. All training images are resized to this dimension (e.g., 640x640 pixels).
- `batch`: The number of images processed together in one forward/backward pass during training. This should be set based on available GPU memory; larger batch sizes can sometimes lead to more stable gradients but require more memory.
- `name`: A custom name for this specific training run. YOLOv8 will create a directory with this name (e.g., `runs/detect/yolov8n_medical_supplies_finetuned`) to store all training artifacts, including model weights, logs, and visualizations.
- `patience`: Enables early stopping. If the validation metrics do not show improvement for a specified number of epochs (here, 5), the training process will halt. This helps prevent overfitting and can save computational resources if the model has converged.

All training results, including the best model weights (`best.pt`), final model weights (`last.pt`), performance metrics (like mAP), confusion matrices, and example validation images with predictions, will be saved in a directory structured as `runs/detect/<name_argument_value>` (e.g., `runs/detect/yolov8n_medical_supplies_finetuned`).

## 5. Inference and Visualization

In [ ]:
# Import necessary libraries for inference and display
from ultralytics import YOLO # YOLO class for model loading and prediction
import cv2                  # OpenCV for image reading and color conversion
import matplotlib.pyplot as plt # Matplotlib for displaying images in the notebook
import os                   # For path manipulation and directory listing

# Path to the best trained model weights.
# This path is determined by the 'name' argument used during 'model.train()'.
# The 'best.pt' file contains the model weights that achieved the best validation performance.
fine_tuned_model_path = 'runs/detect/yolov8n_medical_supplies_finetuned/weights/best.pt'

# Initialize model variable
inference_model = None 

# Check if the trained model file exists
if not os.path.exists(fine_tuned_model_path):
    print(f"Error: Trained model weights not found at {fine_tuned_model_path}")
    print("Please ensure that:")
    print("1. The training process in the previous section completed successfully.")
    print("2. The 'name' argument in model.train() ('yolov8n_medical_supplies_finetuned') matches the path used here.")
    print("3. The 'best.pt' file was generated in the specified directory.")
    # If the model isn't found, inference_model remains None, and subsequent cells should handle this.
else:
    # Load the fine-tuned model for inference
    inference_model = YOLO(fine_tuned_model_path)
    print(f"Successfully loaded fine-tuned model from {fine_tuned_model_path}")
    # Optionally, display model information to confirm it's the correct one
    # inference_model.info()

After training, the model weights that achieved the best performance on the validation set are saved as `best.pt`. We load this file to create our `inference_model`. The path to these weights is typically `runs/detect/<name_used_in_training>/weights/best.pt`. In our specific case, this resolves to `runs/detect/yolov8n_medical_supplies_finetuned/weights/best.pt`.

In [ ]:
# Specify the path to validation images for inference.
# We'll use some images from the validation set of our downloaded 'VOC_subset' dataset.
validation_image_dir = 'VOC_subset/valid/images/' 

# Initialize a list to store paths of sample images for inference
sample_image_files = []

# Check if the validation image directory exists
if os.path.exists(validation_image_dir):
    try:
        # Get a list of image filenames (ending with .png, .jpg, or .jpeg)
        image_filenames = [f for f in os.listdir(validation_image_dir) 
                           if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        # Select the first 3 images for demonstration, if available
        sample_image_files = [os.path.join(validation_image_dir, f) for f in image_filenames[:3]] 
        if not sample_image_files:
            print(f"No images found in {validation_image_dir}")
    except Exception as e:
        print(f"Error listing images from {validation_image_dir}: {e}")
else:
    print(f"Error: Validation image directory not found at '{validation_image_dir}'. Please check the path.")

# Proceed with inference only if the model loaded successfully and sample images were found
if inference_model and sample_image_files:
    print(f"Performing inference on {len(sample_image_files)} sample images...")
    # Run inference using the predict method.
    # 'source' can be a single image path, a list of image paths, or a directory.
    # 'save=True' will save the images with predictions (bounding boxes, labels, scores).
    # 'conf=0.5' sets the confidence threshold; only detections with score > 0.5 will be considered.
    results = inference_model.predict(source=sample_image_files, save=True, conf=0.5) 
    print(f"Inference complete. Results are saved by default in a 'runs/detect/predict*' directory.")
    # The 'results' object itself is a list of Results objects, containing detailed detection data.
elif not inference_model:
    print("Skipping inference as the fine-tuned model was not loaded successfully.")
else:
    print("Skipping inference as no sample images were found.")

The `model.predict()` (or in our case, `inference_model.predict()`) function is used to perform object detection on new images or videos.

**Key `predict()` Parameters:**
- `source`: This can be a path to a single image, a list of image paths, a path to a directory containing images, a path to a video file, or even a live camera feed ID.
- `save`: If set to `True` (default is `False`), the images (or video frames) with the drawn bounding boxes, class labels, and confidence scores will be saved. They are typically stored in a new directory like `runs/detect/predict`, `runs/detect/predict2`, and so on.
- `conf`: This is the confidence threshold. Only detections with a confidence score greater than this value will be considered valid. A common starting point is 0.25, but it can be adjusted (e.g., to 0.5 as used here) depending on the desired trade-off between recall and precision.

In [ ]:
# Display the images with detections.
# We will load and show the images that YOLOv8 saved during the predict(save=True) step.

# Proceed only if the model was loaded and inference was attempted on some images.
if inference_model and sample_image_files:
    # Base directory where YOLOv8 saves prediction runs
    saved_results_base_dir = 'runs/detect/'
    predict_dirs = [] # To store paths of all 'predict*' directories

    # Check if the base results directory exists
    if os.path.exists(saved_results_base_dir):
        # List all directories starting with 'predict' (e.g., predict, predict2, etc.)
        predict_dirs = sorted([
            os.path.join(saved_results_base_dir, d) 
            for d in os.listdir(saved_results_base_dir) 
            if d.startswith('predict') and os.path.isdir(os.path.join(saved_results_base_dir, d))
        ])
    
    # If any 'predict*' directories were found
    if predict_dirs:
        latest_predict_dir = predict_dirs[-1] # Get the path to the most recent prediction directory
        print(f"Showing results from the latest prediction directory: {latest_predict_dir}")
        
        # List all image files within the latest prediction directory
        result_image_paths = [os.path.join(latest_predict_dir, f) 
                              for f in os.listdir(latest_predict_dir) 
                              if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        if result_image_paths:
            # Determine how many images to display (up to 3 or the number available)
            num_images_to_display = min(len(result_image_paths), 3)
            
            plt.figure(figsize=(20, 15)) # Set a larger figure size for better visibility
            for i, img_path in enumerate(result_image_paths[:num_images_to_display]):
                # Read the image using OpenCV
                img = cv2.imread(img_path)
                if img is not None:
                    # Convert image from BGR (OpenCV default) to RGB (Matplotlib default)
                    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
                    
                    # Create a subplot for each image
                    plt.subplot(1, num_images_to_display, i + 1)
                    plt.imshow(img_rgb)
                    plt.axis('off') # Hide axes for cleaner display
                    plt.title(os.path.basename(img_path)) # Show filename as title
                else:
                    print(f"Warning: Could not read image {img_path}")
            plt.tight_layout() # Adjust subplot parameters for a tight layout
            plt.show() # Display the figure with subplots
        else:
            print(f"No result images found in {latest_predict_dir}. Ensure 'model.predict(save=True)' ran correctly.")
    else:
        print(f"No prediction directory (e.g., 'runs/detect/predict*') found. Make sure 'model.predict(save=True)' was executed.")
else:
    print("Skipping display of results as either the model was not loaded or no images were processed for inference.")

The code cell above is responsible for visualizing the detection results. It works as follows:
1.  It locates the most recent prediction directory created by `inference_model.predict(save=True)` (e.g., `runs/detect/predict`, `runs/detect/predict2`, etc.).
2.  It lists the image files (JPG, PNG) within that directory.
3.  It then uses `matplotlib.pyplot` to display up to the first 3 of these images in the notebook output.
4.  OpenCV (`cv2`) is used to read the images, and they are converted from BGR (OpenCV's default color order) to RGB (Matplotlib's expected color order) for correct color display.

## 6. Conclusion

This notebook provided a step-by-step guide to performing object detection using YOLOv8. We have successfully:

1.  **Set up the environment** by installing `ultralytics`, `torch`, and `opencv-python-headless`.
2.  **Prepared the data** by downloading the "Objects Medical Supplies" dataset from Roboflow, inspecting its structure, and creating a `custom_data.yaml` file with appropriate paths for training.
3.  **Fine-tuned a pre-trained YOLOv8n model** on our custom dataset for 25 epochs, leveraging transfer learning.
4.  **Performed inference** using the fine-tuned model on sample validation images and visualized the detection results, including bounding boxes and class labels.

### Potential Next Steps:

- **Experiment with different YOLOv8 models:** Try larger models like `yolov8s.pt` or `yolov8m.pt` for potentially higher accuracy, or explore specialized models if available.
- **Train for more epochs:** Increase the number of training epochs (e.g., 100, 200, or more) and monitor validation metrics to find the optimal training duration and avoid overfitting (early stopping with `patience` helps here).
- **Use a larger or different dataset:** Apply this workflow to other datasets, or expand the current dataset with more images and annotations.
- **Hyperparameter tuning:** Experiment with different learning rates, batch sizes, image sizes (`imgsz`), and data augmentation techniques to optimize model performance.
- **Evaluate model performance:** Use YOLOv8's evaluation mode (`model.val()`) to get detailed metrics like mAP (mean Average Precision) on a test set to quantitatively assess the model's accuracy.
- **Export and deploy the model:** Export the trained model to formats like ONNX or TensorRT for deployment in various applications or edge devices.
- **Explore other YOLOv8 features:** Investigate other capabilities of YOLOv8, such as segmentation, classification, or pose estimation, if your task requires them.